# Some setup stuff

Here we will be setting up some stuff that will be helpful going forward, such as the location of Bath and the time of the Vernal Equinox of the year 2022.

In [42]:
bath_latitude = 51.37
bath_longitude = -2.36

In [43]:
import datetime
import re

today = datetime.datetime.now()
Y2022_equinox = datetime.datetime(2022, 3, 20, 15, 33)
time_since_equinox = str(today-Y2022_equinox)
days_passed_since_equinox = int(re.split(' days', time_since_equinox)[0])

## Increasing the RA

The Right Ascension in most databases is measured at the vernal equinox, the 20th of March. However, since the Earth moves around the Sun it is not the same all year around. Each day the Earth covers 360/365 degrees of its rotation, and each hour, since it rotates a full 360º it covers 15º. As such, each day the RA increases

$$\Delta RA = \frac{360}{365*15}$$

This cell will end up not being relevant to the calculations, but it is still useful to understand how RA works. This [video](https://www.youtube.com/watch?v=kejolGai7X8&list=PLX2gX-ftPVXU3BfWLJfmXfGD08cIit5CI&index=9) I have found explains it quite well.

In [44]:
# A year has 365 days, so that each day the Earth covers 360/365 of its rotation
# A day has 24hr and in that time the Earth revolves 360º, so that each hour it covers 15º
# Each day the RA will increase by 360/(365*15) hrs
import numpy as np

increase_RA = 360/(365*15)
hour_ra_increase = np.floor(increase_RA*days_passed_since_equinox)
minute_ra_increase = (hour_ra_increase - np.floor(hour_ra_increase)) * 60
second_ra_increase = ((minute_ra_increase - np.floor(minute_ra_increase)) * 60).round(4)


## The sidereal time

For these observations we use sidereal time. A sidereal day takes a little less than a solar day, 23h 56 min instead of the usual 24h. This is the time it takes for the Earth to fully rotate around its axis. This is useful because any object crosses the upper meridian when the sidereal time is equal to the object's Right Ascension ([source](https://lco.global/spacebook/sky/sidereal-time/)).

A solar day lasts 24h because it takes that amount of time for the same spot in the Earth to be facing the Sun, since not only does the Earth rotate, but it also moves around the Sun.

Here we are simply computing the sidereal time at Bath right now (whenever _now_ is). This is UTC time, which matches the GMT timezone. At the time of writing this (19th of October), the timezone in Bath is GMT + 1, but that will change on the 30th of October when Daylight saving time is implemented, and we will be back in the GMT/UTC timezone.

In [45]:
# Obtain sidereal time
from astropy.coordinates import EarthLocation
from astropy.time import Time
from astropy import units as u

observing_location = EarthLocation(lat=bath_latitude*u.deg, lon=bath_longitude*u.deg)
observing_time = Time(datetime.datetime.utcnow(), scale='utc', location=observing_location)
LST = observing_time.sidereal_time('mean')

## The sidereal time at the time of observation

Following the same procedure as above, we can compute the sidereal time of the day of observation. I have coded the program to automatically use the day of whenever it's ran on, but it can be changed to any random date. Since we are currently UTC + 1 I have coded the observation to start at 19:15 (20:15 Bath time) and finishing at 1:00 (2:00 Bath time). This has to be changed manually in the 30th, and an hour added to both cases.

Once we have the sidereal times of both the start and the end, we can use it to find which stars are visible.

In [46]:
import datetime
from astropy.time import Time
import re

date_today = datetime.datetime.today().strftime('%Y-%m-%d')
date_tomorrow = (datetime.datetime.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
times = [f'{date_today}T19:15:00', f'{date_tomorrow}T1:00:00']# In UTC, change on 30th October

t = Time(times, format='isot', scale='utc')

In [47]:
start_end_times = re.split(' ',str(t.sidereal_time('apparent', bath_longitude)))
start_sidereal_time = start_end_times[0].strip('[]')
end_sidereal_time = start_end_times[1].strip('[]')

## Getting the right ascension of all stars of our query

Here we are importing the dataframe with the stars of the Monson et al paper, which I saved as a pickle in that notebook. We also split the Right Ascension column into hours minutes and seconds to make calculations later on a bit simpler. With such a small dataset, the impact on performance is not huge, and it makes things much easier.

In [48]:
import pandas as pd
import re
simbad_df = pd.read_pickle('Monson_stars_simbad.pkl')
simbad_df['RA hours'] = simbad_df['RA'].str[0:2]
simbad_df['RA mins'] = simbad_df['RA'].str[3:5]
simbad_df['RA seconds'] = simbad_df['RA'].str[5:]


## Getting the declination of all stars of our query

Same thing as before, but now we do it with the declination column. It's split into three: degrees, minutes and seconds.

In [49]:
simbad_df['DEC degrees'] = simbad_df['DEC'].str[0:3].astype(float)
simbad_df['DEC minutes'] = simbad_df['DEC'].str[4:6].astype(float)
simbad_df['DEC seconds'] = simbad_df['DEC'].str[7:].astype(float)

# Finding the hour angle of different stars

In order to actually compute which stars are visible we are using the hour angle, i.e. how far away from the meridian it is at the beginning and end of our observations. In order to calculate the hour angle we are using the following formula:

$$\mathrm{LHA}_{\mathrm{object}} = \mathrm{LST}-\alpha_{\mathrm{object}}$$

where $\mathrm{LHA}_{\mathrm{object}}$ is the Local Hour Angle of the given object, LST is the local sidereal time and $\alpha_{\mathrm{object}}$ is the Right Ascension of the object. We split it into hours, minutes and seconds and then put it back together.

In [50]:
start_sidereal_time_hour = float(re.split('h', str(start_sidereal_time))[0])
start_sidereal_time_minute = float(re.findall('h.*m', str(start_sidereal_time))[0].strip('hm'))
start_sidereal_time_second = float(re.findall('m.*', str(start_sidereal_time))[0].strip('ms'))


end_sidereal_time_hour = float(re.split('h', str(end_sidereal_time))[0])
end_sidereal_time_minute = float(re.findall('h.*m', str(end_sidereal_time))[0].strip('hm'))
end_sidereal_time_second = float(re.findall('m.*', str(end_sidereal_time))[0].strip('ms'))

In [51]:
simbad_df['Start hour angle hours'] = simbad_df['RA hours'].astype(float) - start_sidereal_time_hour
simbad_df['Start hour angle minutes'] = simbad_df['RA mins'].astype(float) - start_sidereal_time_minute
simbad_df['Start hour angle seconds'] = simbad_df['RA seconds'].astype(float) - start_sidereal_time_second

simbad_df['End hour angle hours'] = simbad_df['RA hours'].astype(float) - end_sidereal_time_hour
simbad_df['End hour angle minutes'] = simbad_df['RA mins'].astype(float) - end_sidereal_time_minute
simbad_df['End hour angle seconds'] = simbad_df['RA seconds'].astype(float) - end_sidereal_time_second

# Finding the observable stars



## Circumpolar star

A circumpolar star never sets, but it rotates around the celestial north pole. The amount of circumpolar stars depends on latitude. There are, for example, no circumpolar stars in the equator, and all of the stars are circumpolar at the poles. A star will be circumpolar if
$$ \theta + \delta > 90º$$

where $\theta$ is the latitude and $\delta$ is the declination.

In [68]:
circumpolar_stars = simbad_df[simbad_df['DEC degrees'] + bath_latitude > 90]

## Discriminating based on RA and Dec

Aside from the circumpolar stars, we want something that will be close to the Meridian at the time of observation. This means an hour angle of $\pm3$ hrs or less, which amounts to $\pm 45$º in the East to West plane. Each hour it will move 15º, so that something that starts with an HA = 3hrs at 8pm will cross the meridian at 11pm. Using the hour angle allows us to circumvent calculating which stars are not observable in this coordinate, since anything not visible will have an hour angle too large.

In terms of declination, we want something that is visible from Bath. Since we are not at the equator, the stars will not be where its declination says (in the North to South direction), but instead the following correction must be applied:

$$\mathrm{Dec}_{\mathrm{Bath}} = \mathrm{Dec} - \mathrm{lat}$$

This will tell us what angle the star has with respect to the zenith. To avoid light pollution and trees we can limit the observable declination to 30º in each direction, so that we are left with a small circle in the sky with our observable stars.

In [69]:
ra_observable = simbad_df[(simbad_df['Start hour angle hours'] < 3) & (simbad_df['Start hour angle hours'] > -3)]
dec_observable = ra_observable[(ra_observable['DEC degrees']-bath_latitude < 30) & (ra_observable['DEC degrees']-bath_latitude > -30)]
dec_observable = pd.concat([dec_observable, circumpolar_stars])
# Here I just get rid of the columns I made for the operations
dec_observable = dec_observable.drop(columns = ['Start hour angle hours', 'Start hour angle minutes', 'Start hour angle seconds', 'End hour angle hours', 'End hour angle minutes', 'End hour angle seconds', 'RA hours', 'RA mins', 'RA seconds', 'DEC degrees', 'DEC minutes', 'DEC seconds'])


In [71]:
# Drop the duplicates in case there are any
dec_observable = dec_observable.drop_duplicates()

In [72]:
dec_observable

,MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,...,Distance_unit,Distance_merr,Distance_perr,Distance_method,Distance_bibcode,FLUX_K,FLUX_G,FLUX_R,FLUX_B,SCRIPT_NUMBER_ID
0,V* UY Cyg,20 56 28.3030,+30 25 40.327,14,14,0.0102,0.0154,90,A,O,...,pc,-24.0161,24.0161,paral,2020yCat.1350....0G,9.715,11.017679,NaN,11.37,1
0,V* XZ Cyg,19 32 29.3050,+56 23 17.494,14,14,0.0135,0.0147,90,A,O,...,pc,-5.8411,5.8411,paral,2020yCat.1350....0G,8.884,9.913754,NaN,9.94,1
0,V* RR Lyr,19 25 27.9129,+42 47 03.693,14,14,0.0224,0.0242,90,A,O,...,pc,-1.6688,1.6688,paral,2020yCat.1350....0G,6.648,7.618805,7.6,7.36,1
0,V* AV Peg,21 52 02.7945,+22 34 29.376,14,14,0.0144,0.0134,90,A,O,...,pc,-8.0166,8.0166,paral,2020yCat.1350....0G,9.322,10.531158,NaN,10.07,1
0,V* RZ Cep,22 39 13.1768,+64 51 30.611,14,14,0.0108,0.0099,90,A,O,...,pc,-2.0294,2.0294,paral,2020yCat.1350....0G,7.968,9.294270,9.3,9.48,1
0,V* SU Dra,11 37 56.6071,+67 19 47.060,14,14,0.0115,0.0117,90,A,O,...,pc,-8.1150,8.1150,paral,2020yCat.1350....0G,8.821,9.784324,NaN,9.37,1
0,V* SW Dra,12 17 46.6283,+69 30 38.233,14,14,0.0169,0.0159,90,A,O,...,pc,-16.9324,16.9324,paral,2020yCat.1350....0G,9.226,10.488967,NaN,10.87,1
0,V* TT Lyn,09 03 07.7864,+44 35 08.119,14,14,0.0127,0.0102,90,A,O,...,pc,-7.6026,7.6026,paral,2020yCat.1350....0G,8.544,9.853911,NaN,9.75,1
0,V* AB UMa,12 11 14.5890,+47 49 43.794,14,14,0.0084,0.0123,90,A,O,...,pc,-19.2301,19.2301,paral,2020yCat.1350....0G,9.576,10.817328,NaN,11.42,1
0,V* RV UMa,13 33 18.0838,+53 59 14.598,14,14,0.0106,0.0120,90,A,O,...,pc,-15.3790,15.3790,paral,2020yCat.1350....0G,9.816,10.858340,NaN,10.97,1
